<a href="https://colab.research.google.com/github/seungmin001/prac_deeplearningzerotoall/blob/main/Lab6_Softmax_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# multinomial classification
import tensorflow as tf
import numpy as np

# Data
x_data= np.array([ # 4 features, 8 data
    [1,2,1,1],
    [2,1,3,2],
    [3,1,3,4],
    [4,1,5,5],
    [1,7,5,5],
    [1,2,5,6],
    [1,6,6,6],
    [1,7,7,7]
], dtype=np.float32)

y_data = np.array([ # 3 label, 8 data
    [0,0,1],
    [0,0,1],
    [0,0,1],
    [0,1,0],
    [0,1,0],
    [0,1,0],
    [1,0,0],
    [1,0,0],
], dtype=np.float32)

nb_classes = 3 # num of classes


# Weight, bias
W = tf.Variable(tf.random.normal([4,3]), name='Weight')
b = tf.Variable(tf.random.normal([1]), name='bias')


########
# raw code

# hypothesis = 일차함수 W*X + b, softmax로 y값 확률으로 바꾸기
hypothesis = tf.nn.softmax(tf.matmul(x_data,W)+b)
print(hypothesis)

# cost function : cross entropy
cost = tf.reduce_mean( - tf.reduce_sum(y_data * tf.math.log(hypothesis), axis=1))
print(cost)

# optimizer (Gradient Descent에 사용되는 도구)
optimizer= tf.keras.optimizers.SGD(learning_rate=0.01)


tf.Tensor(
[[8.2298875e-01 2.5095740e-02 1.5191555e-01]
 [9.1766232e-01 7.9036849e-03 7.4433997e-02]
 [9.8742515e-01 4.0674480e-03 8.5074054e-03]
 [9.9658465e-01 4.3234703e-04 2.9830153e-03]
 [9.9649554e-01 1.4000380e-05 3.4904887e-03]
 [9.9320716e-01 2.0272324e-03 4.7656028e-03]
 [9.9778038e-01 1.8904004e-05 2.2007278e-03]
 [9.9912912e-01 3.5490773e-06 8.6738431e-04]], shape=(8, 3), dtype=float32)
tf.Tensor(4.296997, shape=(), dtype=float32)


In [3]:
###############################################################################
# 함수 형태로 작성

# softmax hypothesis
def hypothesis(X): 
    base = tf.matmul(X,W) + b
    return tf.nn.softmax(base)

hypothesis(x_data)

<tf.Tensor: shape=(8, 3), dtype=float32, numpy=
array([[8.2298875e-01, 2.5095740e-02, 1.5191555e-01],
       [9.1766232e-01, 7.9036849e-03, 7.4433997e-02],
       [9.8742515e-01, 4.0674480e-03, 8.5074054e-03],
       [9.9658465e-01, 4.3234703e-04, 2.9830153e-03],
       [9.9649554e-01, 1.4000380e-05, 3.4904887e-03],
       [9.9320716e-01, 2.0272324e-03, 4.7656028e-03],
       [9.9778038e-01, 1.8904004e-05, 2.2007278e-03],
       [9.9912912e-01, 3.5490773e-06, 8.6738431e-04]], dtype=float32)>

In [4]:
# cost func : cross-entropy
def cost_func(X,Y):
    logits=hypothesis(X)
    cost_raw = - tf.reduce_sum( Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost_raw)
    return cost_mean

cost_func(x_data,y_data)

<tf.Tensor: shape=(), dtype=float32, numpy=4.296997>

In [5]:

# gradient
def grad_func(X,Y):
    with tf.GradientTape() as tape:
        cost=cost_func(X,Y)
        grads=tape.gradient(cost, [W,b])
        return grads

grad_func(x_data, y_data)

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
 array([[ 1.4491925 , -0.74288774, -0.7063047 ],
        [ 1.6862648 , -1.2416394 , -0.4446255 ],
        [ 2.6812844 , -1.8658106 , -0.81547356],
        [ 2.8141556 , -1.9910368 , -0.8231187 ]], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>]

In [6]:

# optimizer (Gradient Descent에 사용되는 도구)
optimizer= tf.keras.optimizers.SGD(learning_rate=0.01)

epoch = 5000

for step in range(epoch+1):
    # cost계산
    # with tf.GradientTape() as tape:
    #    cost = tf.reduce_mean( - tf.reduce_sum(y_data * tf.math.log(hypothesis), axis=1))
        
    # cost 함수에 대한 W,b의 기울기 
    # grads = tape.gradient(cost, variables)

    grads=grad_func(x_data, y_data)
    # 기울기 gradient descent 적용
    optimizer.apply_gradients(zip(grads,[W,b]))

    if step % 500 == 0:
        print("{:5} | {:12.5f}".format(step, cost_func(x_data,y_data)))


    0 |      3.98166
  500 |      0.70948
 1000 |      0.62454
 1500 |      0.57770
 2000 |      0.54545
 2500 |      0.52064
 3000 |      0.50028
 3500 |      0.48283
 4000 |      0.46746
 4500 |      0.45363
 5000 |      0.44100


In [7]:
#prediction

sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.02959524 0.28181067 0.68859416]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [9]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[5.4446836e-03 3.1460948e-02 9.6309441e-01]
 [1.7174548e-02 2.1045373e-01 7.7237171e-01]
 [4.4500301e-04 3.9921945e-01 6.0033548e-01]
 [3.5709579e-04 5.9840173e-01 4.0124124e-01]
 [5.5136400e-01 4.1116983e-01 3.7466154e-02]
 [1.7228895e-01 8.2768089e-01 3.0095776e-05]
 [6.8552101e-01 3.1378496e-01 6.9404894e-04]
 [8.0804318e-01 1.9184546e-01 1.1139441e-04]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 0 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
